# Práctica 4 - Simulación de Montecarlo

## Problema - Simulación a cholón


### Parte 1: La potencia de un test

En este ejercicio vamos a calcular la potencia de un test de hipótesis. Definimos la potencia del test $T$ como la probabilidad de obtener un resultado positivo (p.ej. un p-valor < 0.05) cuando verdaderamente hay efecto. Esto es, si $T$ contrasta la hipótesis nula $H_0$ contra la alternativa $H_1$ con un nivel de significación $\alpha$, la **potencia de $T$** es:
$$\mathrm{pow}(T):=P \left (p_T < \alpha | H_1 \mathrm{is\ true} \right )$$
Dicho de otro modo, nos mide la tasa de verdaderos positivos (y, por ende, la tasa de falsos positivos, que es la que se suele reportar más a menudo). Incluso cuando los datos subyacentes son normales, no hay una manera directa de obtener la potencia de un test, tal y como sí ocurre con la distribución del estadístico de contraste; así que no queda más remedio que aproximarla con un algoritmo de simulación.

Supón que quieres comparar la tasa de conversión (CR) de dos productos similares en una web de e-commerce. Tras realizar un test AB, se obtiene que la CR de A es 0.031 y la CR de B es 0.054. Si las conversiones siguen sendas distribuciones binomiales y hemos tenido 4568 y 5021 usuarios únicos viendo cada producto, calcula la potencia del test que compara las dos proporciones si rechazamos la hipótesis nula con un p-valor de 0.05.

*Indicación*: repite el test muchas veces, cada una simulando las dos binomiales aleatoriamente, y anota cuántas veces obtienes el p-valor deseado o mejor. Prueba con distinto número de iteraciones y comenta los resultados. Aquí tienes un snippet de código para generar muestras binomiales de probabilidad `p` y tamaño `k` a partir de repetir una Bernoulli.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from scipy.stats import bernoulli

# Modifica los valores y observa los resultados que obtienes en distintas pruebas

k = 100
p = 0.4

binom_sample = bernoulli.rvs(p, size=k)

print(binom_sample)

[0 0 0 1 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 1 0 1 0 1 0 0 0 1 1 1 0 1 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0
 1 0 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0]


Guardamos las dos *conversions rates* y el número de usuarios:

In [ ]:
CR_A = 0.031
CR_B = 0.054
size_A = 4568
size_B = 5021

Generamos una distribución para cada par de variables:

In [ ]:
binom_A = bernoulli.rvs(CR_A, size = size_A)
np.sum(binom_A) / size_A

0.029115586690017514

In [ ]:
binom_B = bernoulli.rvs(CR_B, size = size_B)
np.sum(binom_B) / size_B

0.05696076478789086

Parece que la tasa de conversión del producto B es mejor. Vamos a realizar inferencia para combrobarlo. 

Vamos a hacer un test de proporciones para ver si las medias son significativamente distintas. Las hipótesis, como siempre, son: 
$$ H_0: P_{A} = P_{B}, $$
$$ H_1: P_{A} \neq P_{B} $$

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

count = sum(binom_A), sum(binom_B)
nobs = len(binom_A), len(binom_B)

proportions_ztest(count, nobs, alternative = 'two-sided')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


(-6.661977142639641, 2.701680541458435e-11)

Podemos asegurar que las proporciones son distintas.

Vamos a ver si podemos asegurar que la tasa de conversión B es mejor que la A. Las hipótesis son:
$$ H_0: P_{A} \geq P_{B}, $$
$$ H_1: P_{A} < P_{B} $$

In [ ]:
count = sum(binom_A), sum(binom_B)
nobs = len(binom_A), len(binom_B)

proportions_ztest(count, nobs, alternative = 'smaller')

(-6.661977142639641, 1.3508402707292175e-11)

El p-valor es muy pequeño, por lo que podemos rechazar la hipótesis nula y confirmar que la tasa de conversión de B es mejor que la de A.

Queremos ver ahora que potencia tiene el test que acabamos de realizar. Vamos a repetir el test varias veces y aplicar la fórmula de probabilidad para ver cuántas veces nos da este mismo resultado.

In [ ]:
# Creamos un data frame para guardar los p-valores
p_values = pd.DataFrame()

# Hacemos el bucle
for i in range(0, 100):    
  binom_A = bernoulli.rvs(CR_A, size = size_A)
  binom_B = bernoulli.rvs(CR_B, size = size_B)
  count = np.array([sum(binom_A), sum(binom_B)])
  nobs = np.array([len(binom_A), len(binom_B)])
  stat, p_val = proportions_ztest(count, nobs, alternative = 'smaller')
  data = {
            "p-val": [p_val],
         }
  p_values = pd.concat([p_values, pd.DataFrame(data)], sort = False)

Mostramos el data frame generado:

In [ ]:
p_values

,p-val
0,1.079767e-14
0,2.549014e-09
0,2.397748e-08
0,3.423222e-12
0,2.969072e-06
...,...
0,4.526709e-08
0,2.862433e-04
0,1.982264e-09
0,3.430473e-07


Así pues, la potencia de nuestro test es:

In [ ]:
favorable_cases = len(p_values[p_values.iloc[:, 0] < 0.05])
possible_cases = len(p_values)
print(float(favorable_cases) / float(possible_cases))

1.0


### Parte 2: ¿Cuántos usuarios necesito para estar seguro de los resultados?

El tema de la potencia lleva al razonamiento siguiente: dado que un test podría no ser suficientemente potente, ello implica que es relativamente frecuente obtener p-valores bajos cuando en realidad no hay efecto. Esto debería suceder, sobre todo, cuando las tasas de conversión son bajas y disponemos de pocas muestras. En tal caso, ¿cuantos usuarios se necesitan para que los resultados del test sean fiables?

Esta no es una cuestión baladí, y la realidad es que a menudo es muy complicado reportar resultados de un test AB porque, o bien se desconoce el tamaño muestral necesario para hacerlo fiable, o bien dicho tamaño es tan grande que es imposible de alcanzar.

La norma no escrita en la industria es que, para reportar resultados de un test AB, se necesita un p-valor de 0.05 o inferior y que el test tenga una potencia de, al menos, 0.8 (80%). Fijadas estas dos cantidades, simula tests de proporciones con los datos de la parte 1. Comienza con pocas muestras por grupo (por ejemplo, 100 en cada variante) y ves aumentando los tamaños muestrales hasta que obtengas una N que dé un p-valor de 0.05 o menos y una potencia de 0.8 o más. ¿Qué tamaño muestral se necesita?

*Indicación*: A cada iteración de tamaño muestral necesitas simular los p-valores para saber la potencia; por lo tanto tendrás que hacer un loop de loops. Dependiendo de la CPU y la RAM de tu máquina esto puede ser bastante lento. Ten paciencia, deja la máquina varias horas calculando si es necesario. Tal cosa es habitual en entornos de trabajo y experimentación en Data Science.

Definimos una función con el procedimiento anterior:

In [ ]:
def potencia(n, CR_A, CR_B, alfa):
  p_values = pd.DataFrame()
  for i in range(0, 100):
    binom_A = bernoulli.rvs(CR_A, size = n)
    binom_B = bernoulli.rvs(CR_B, size = n)
    count = np.array([sum(binom_A), sum(binom_B)])
    nobs = np.array([len(binom_A), len(binom_B)])
    stat, p_val = proportions_ztest(count, nobs, alternative = 'smaller')
    data = {
              "p-val": [p_val],
          }
    p_values = pd.concat([p_values, pd.DataFrame(data)], sort = False)
  
  favorable_cases = len(p_values[p_values.iloc[:, 0] < alfa])
  possible_cases = len(p_values)
  P = float(favorable_cases) / float(possible_cases)
  
  p_val = float(p_values.mean())
  return P, p_val

Podemos comprobar que funciona:

In [ ]:
potencia(100, CR_A, CR_B, 0.05)

(0.18, 0.28312684237662805)

Podemos definir otra función con otro bucle que calcule el tamaño muestral necesario para conseguir un p-valor de 0.05 o menos y una potencia de 0.8 o más:

In [ ]:
def tamaño_muestral(CR_A, CR_B, alfa, pot):
  n = 100
  P = 0
  p_val = 1

  while True:
    result = potencia(n, CR_A, CR_B, alfa)
    P = result[0]
    p_val = result[1]
    print(n, P, p_val)
    if P > pot and p_val < alfa: break
    n = n + 50

La llamamos con nuestros datos:

In [ ]:
tamaño_muestral(CR_A, CR_B, 0.05, 0.8)

100 0.26 0.26403519811803833
150 0.2 0.24673739241793782
200 0.36 0.21002052858744322
250 0.37 0.19173869230267068
300 0.34 0.16685108499209414
350 0.46 0.15140465828276317
400 0.49 0.14054665553107515
450 0.48 0.11141576989909532
500 0.51 0.109632832367677
550 0.63 0.08811950963064037
600 0.65 0.06609506704125467
650 0.69 0.06488761117816329
700 0.68 0.048755806273215914
750 0.72 0.0446370221373098
800 0.69 0.05980447632070105
850 0.82 0.041689034622211685


El tamaño muestral necesario para este test estaría alrededor de los 900 registros.